In [1]:
import os
import csv
import numpy as np
from astropy.io import fits

# === USER INPUT ===
input_path = "/home/rebeccaz/Data/20230919_VAMPIRES_Internal_Cal/collapsed/"  # <-- change this
output_csv = os.path.join(input_path, "2023_09_19_Collapsed_Data.csv")  # <-- change this

# === Collect FITS Files ===
def collect_fits_files(path):
    if os.path.isfile(path) and path.lower().endswith(".fits"):
        return [path]
    elif os.path.isdir(path):
        return sorted([
            os.path.join(path, f)
            for f in os.listdir(path)
            if f.lower().endswith(".fits")
        ])
    else:
        raise ValueError("Input must be a .fits file or a directory containing FITS files.")

# === Extract Headers and Flux ===
def extract_fits_headers_and_flux(filepaths):
    headers_list = []
    all_keys = set()

    for filepath in filepaths:
        try:
            with fits.open(filepath) as hdul:
                header = dict(hdul[0].header)
                data = hdul[0].data

                total_flux = ""

                if isinstance(data, np.ndarray) and data.ndim >= 2:
                    obs_mode = header.get("OBS-MOD", "").strip().upper()

                    if obs_mode == "IPOL_MBI" and data.ndim == 3:
                        print("Entered MBI")
                        # Sum each frame separately along the third axis
                        total_flux = [np.sum(data[i, :, :][np.isfinite(data[i, :, :])]) for i in range(data.shape[0])]
                    else:
                        # Sum all finite values in the entire array
                        total_flux = np.sum(data[np.isfinite(data)])

                filename = os.path.basename(filepath)
                headers_list.append((filename, total_flux, header))
                all_keys.update(header.keys())

        except Exception as e:
            print(f"Could not read {filepath}: {e}")
            headers_list.append((os.path.basename(filepath), "", {}))

    return sorted(headers_list, key=lambda x: x[0]), sorted(all_keys)

# === Write to CSV ===
def write_csv(headers_list, all_keys, output_path):
    with open(output_path, "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["FILENAME", "TOTAL_FLUX"] + list(all_keys))
        for filename, total_flux, header in headers_list:
            # Convert list of fluxes to string if needed
            if isinstance(total_flux, (list, np.ndarray)):
                flux_str = "[" + ", ".join(f"{v:.3f}" for v in total_flux) + "]"
            else:
                flux_str = total_flux
            row = [filename, flux_str] + [header.get(key, "") for key in all_keys]
            writer.writerow(row)

# === Run Everything ===
fits_files = collect_fits_files(input_path)
headers_list, all_keys = extract_fits_headers_and_flux(fits_files)
write_csv(headers_list, all_keys, output_csv)

print(f"Wrote FITS headers and total flux for {len(headers_list)} file(s) to:\n{output_csv}")


Wrote FITS headers and total flux for 1280 file(s) to:
/home/rebeccaz/Data/20230919_VAMPIRES_Internal_Cal/collapsed/2023_09_19_Collapsed_Data.csv
